In [1]:
import sys
sys.path.append("..")

In [2]:
import os
from models.duration_model import DurationModel
import tensorflow as tf
import numpy as np

# Define parametes

In [ ]:
# train_parameters = {
#     "lr": 0.0001,
#     "decay_steps": 300,
#     "decay_rate": 0.9,
# }

# model_parameters = {
#     "embedding_size": 256,
#     "num_dense_layers": 2,
#     "dense_layers_units": 256,
#     "num_bidirectional_layers": 4,
#     "num_bidirectional_units": 254,
#     "dropout_prob": 0.2,
#     "buckets": 100
# }

# input_vocab_size= 50 # TODO: Define


# num_steps = 2

# save_energy = 2
train_parameters = {
    "lr": 0.0001,
    "decay_steps": 300,
    "decay_rate": 0.9,
}

model_parameters = {
    "embedding_size": 16,
    "num_dense_layers": 2,
    "dense_layers_units": 16,
    "num_bidirectional_layers": 2,
    "num_bidirectional_units": 16,
    "dropout_prob": 0.2,
    "buckets": 10
}

input_vocab_size= 50 # TODO: Define

num_steps = 2

save_energy = 2

# Train model

In [ ]:
with tf.Session() as sess:
    X = tf.placeholder(tf.int32, [None, 200])
    X_seq_len = tf.placeholder(tf.int32, [None])
    Y = tf.placeholder(tf.int32, [None, 200])

    model = DurationModel(
        X, X_seq_len,
        input_vocab_size, Y, model_parameters,
        train_parameters
    )

    train_writer = tf.summary.FileWriter('../log/train_grapheme_to_phoneme_model_notebook/train', sess.graph)
    
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    saver = tf.train.Saver(max_to_keep=3, keep_checkpoint_every_n_hours=3)
    
    for _ in xrange(num_steps):
        
        target_inputs = np.ones((2, 200))
        target_inputs_seq_len = np.ones(2)
        target_outpus = np.zeros((2, 200))
        
        out = sess.run([
            model.train_op,
            model.global_step,
            model.logits,
            model.transition_params,
            model.loss,
            model.summary
        ], feed_dict={X: target_inputs, X_seq_len: target_inputs_seq_len, Y: target_outpus})
        _, global_step, logits, transition_params, loss, summary  = out
        
        print global_step
        print logits
        print transition_params
        print loss


        train_writer.add_summary(summary, global_step)

        # detect gradient explosion
        if loss > 1e8 and global_step > 500:
            print('loss exploded')
            break

        if global_step % save_energy == 0 and global_step != 0:

            print('saving weights')
            if not os.path.exists('../weights/train_grapheme_to_phoneme_model_notebook/'):
                os.makedirs('../weights/train_grapheme_to_phoneme_model_notebook/')
            saver.save(sess, '../weights/train_grapheme_to_phoneme_model_notebook/', global_step=global_step)
    
    coord.request_stop()
    coord.join(threads)

/Users/josebermeo/miniconda2/envs/deep_voice_2/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1
[[[-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]
  [-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]
  [-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]
  ..., 
  [-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]
  [-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]
  [-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]]

 [[-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]
  [-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]
  [-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]
  ..., 
  [-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]
  [-0.07859102  0.05210628 -0.30170524 ...,  0.09918177  0.27090827
    0.02621151]
  [-0.07859102  0.05210628 -0.30170524 ...,  0.09918177 